<a href="https://colab.research.google.com/github/sachit-menon/stylegan-encoder/blob/tiedlatents/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/stylegan-refactor

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/stylegan-refactor


In [0]:
!pip install tf-nightly-gpu

In [0]:
import argparse
import tensorflow as tf
import pickle
import numpy as np
import sys
from training.networks_stylegan import G_synthesis
tf.logging.set_verbosity(tf.logging.INFO)

def my_model(features, labels, mode, params):
  latent = tf.get_variable("latent",shape=(1,18,512),dtype='float32',initializer=tf.initializers.random_normal())
  with tf.variable_scope("GAN",reuse=tf.AUTO_REUSE):
    out = G_synthesis(latent,use_noise=False)
        
  tf.train.init_from_checkpoint('StyleGan/model.ckpt',{'G_synthesis_1/': "GAN/"})
  out = tf.transpose(out,(0,2,3,1),name="out_im")
  tf.summary.image('out_summary', out)
  
  loss = tf.reduce_sum(tf.square(out))
  
  optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
  train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step(),var_list=[latent])
  return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

run_config = tf.estimator.RunConfig(save_checkpoints_steps=None,
                                    save_checkpoints_secs=None)

classifier = tf.estimator.Estimator(
    model_fn=my_model,model_dir = './tf',config = run_config)

classifier.train(
    input_fn=lambda: None,
    steps=2000)

print("trained")

In [0]:
%reload_ext tensorboard
%tensorboard --logdir tf